# Jupyter SCRAPbook to test custom activation function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras.models
import tensorflow as tf
import pandas as pd
import csv
from src.functions import initializers as ci # i.e., custom initializers
from src.functions.layers import ActivLin1D
from src.functions import activation_parameters as ap
from src.utils import data_handler as dh
import matplotlib.pyplot as plt

In [139]:
# Model save path:
inv_save_path = '/Users/francescomaraschin/Desktop/IntelligentVehicles/Project_NN_Conda/data/trained_models/inverse_model/inverse_model.h5'

inverse_model = tf.keras.models.load_model(filepath=inv_save_path,
                                            custom_objects={
                                                'MyInitializer': ci.MyInitializer,
                                                'ActivLin1D': ActivLin1D
                                            })

# Reshape and split data
inverse_dataset_path = '/Users/francescomaraschin/Desktop/IntelligentVehicles/LongitudinalControllerNN/Data/csv/inverse_dataset.csv'
inverse_dataset = dh.load_csv(inverse_dataset_path)
train_data, valid_data, time = dh.window_data(dataset=inverse_dataset,
                                              input_labels=['acceleration',
                                                            'velocity'],
                                              output_labels=['pedal'],
                                              input_window=10,
                                              output_window=1,
                                              batch_size=100,
                                              validation_split=0.3)

velocity = train_data[0][1]

In [140]:
np.shape(velocity)

(11200, 10)

## Activation function

In [141]:
def activ_lin_1d(act_in, chan_id, chan_arr):
    '''
    Copy of the ActivLin1D class as Python function using numpy covering only the actual channels.

    Arguments:
     - act_in:   the input that activates the channels;
     - chan_id:  index of the channel center (i.e., of the chan_arr element);
     - chan_arr: array containing the channel centers.

    Outputs:
     - act_fcn: the activation function.
    '''

    # Compute the number of channels
    chan_num = len(chan_arr)

    # First dimension of activation
    if chan_id == 0:
        if chan_num != 1:
            ampl    = chan_arr[1] - chan_arr[0]
            act_fcn = np.minimum(
                np.maximum(-(act_in - chan_arr[0])/ampl + 1, 0), 1)

        else:
            act_fcn = 1 # in case the user only wants one channel

    elif chan_id != 0 and chan_id == (chan_num - 1):
        ampl    = chan_arr[-1] - chan_arr[-2]
        act_fcn = np.minimum(
            np.maximum((act_in - chan_arr[-2])/ampl, 0), 1)

    else:
        ampl_1  = chan_arr[chan_id] - chan_arr[chan_id - 1]
        ampl_2  = chan_arr[chan_id + 1] - chan_arr[chan_id]
        act_fcn = np.minimum(
            np.maximum((act_in - chan_arr[chan_id - 1])/ampl_1, 0),
            np.maximum(-(act_in - chan_arr[chan_id])/ampl_2 + 1, 0))

    return act_fcn

## Test

In [176]:
num_chan = 10
chan_arr = np.array([-1.0,
        -0.8114791181352403,
        -0.6229582362704806,
        -0.43443735440572107,
        -0.24591647254096138,
        -0.0573955906762017,
        0.13112529118855787,
        0.31964617305331755,
        0.5081670549180772,
        0.6966879367828369
                     ])

act_in = velocity[0]

act_fcn  = np.array([activ_lin_1d(act_in=act_in,
                                  chan_id=i,
                                  chan_arr=chan_arr) for i in range(num_chan)]).T

In [177]:
np.shape(act_in)
act_in

array([-2.9426666e-09, -2.2764150e-08,  2.0929241e-08, -3.6351227e-07,
       -6.3535567e-06, -5.4394652e-05, -4.0748707e-04, -1.8456370e-03,
        4.7330112e-05,  1.7191232e-03], dtype=float32)

In [175]:
act_fcn

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [148]:
np.shape(act_fcn)

(10, 10)

In [97]:
act_fcn

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.6955478 , 0.30445215, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.69554794, 0.30445206, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.6955477 , 0.3044523 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.6955497 , 0.30445024, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.69558156, 0.30441847, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.6958363 , 0.30416366, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.6977093 , 0.30229068, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [76]:

pd.DataFrame(act_fcn[0]).to_csv("/Users/francescomaraschin/Desktop/IntelligentVehicles/LongitudinalControllerNN/NN_Python/test/act_fcn.csv",
                              header=None,
                              index=None ,
                              lineterminator = '\n'
                              )

In [49]:
check_sum =  np.mean(np.sum(act_fcn, axis = 0))
assert check_sum == 1.0,\
           f'The sum of all activation functions is not 1, it is: {check_sum:.4e}'
print(f'The sum of all activation functions is: {check_sum:.4e}')

The sum of all activation functions is: 1.0000e+00
